In [1]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="mydb",
    user="myuser",
    password="mypassword",
    host="localhost",  # or "db" if you're running inside another container
    port="5432"
)

# Create a cursor
cursor = conn.cursor()

# Create a table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL
    )
""")

# Insert data
cursor.execute("INSERT INTO users (name) VALUES (%s)", ("Alice",))
cursor.execute("INSERT INTO users (name) VALUES (%s)", ("Bob",))
conn.commit()

# Query data
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Clean up
cursor.close()
conn.close()


(1, 'Alice')
(2, 'Bob')
